# Checkpoint 2
### Due: Tuesday, 8 November, 2022 at 11:00am BST

### Total points: 100

### Read This First

Wherever you see raise NotImplementedError(), remove that line and put your code there.

Put the code that produces the output for a given task in the cell indicated. You are welcome to add as many cells as you like for imports, function definitions, variables, etc.

Your notebook must run correctly when executed once from start to finish. Your notebook will be graded based on how it runs, not how it looks when you submit it. To test this, go to the Kernel menu and select Restart & Run All.

Once you are happy with it, clear the output by selecting Restart & Clear Output from the Kernel menu.

You should aim for your code to not take more than around a minute per task. 

Submit through Noteable.

# Task 1 (20 points)

The first task is to find the number of local minima of the function. 
The function is defined in any number of dimensions. 

And your job is to function the number of local minimima of the function for dimensions 1, 2 3, 4, 5. Your code should return the list or array of 5 integer values.

The number of local minima for ndim = 1 should be 4. All the minimima should be within -10..10 range

We also know that the function has no minima separated by less than 0.01.

In [ ]:
import numpy as np
import scipy.optimize
import scipy.linalg
import matplotlib.pyplot as plt
import scipy.stats
%matplotlib inline

In [ ]:
T1_D = np.load('ch2_task1.npz')['D']
dims = [1, 2, 3, 4, 5]
import scipy.special as ssp

def blackbox_func(p):
    """
    This is is the function that you need to optimize
    DO NOT CHANGE THE CODE INSIDE THIS FUNCTION
    """
    p = np.asarray(p)
    ndim = len(p)
    pos = dims.index(ndim)
    curD = T1_D[10000 * pos:10000 * (pos + 1), :ndim]
    return -ssp.logsumexp(np.sum(-0.5 * ((p[None, :] - curD) / .3)**2, axis=1))

# The function is defined in  1,2,3,4,5 dimensions
print (blackbox_func([0]))
print (blackbox_func([0,0]))
print (blackbox_func([0,0,0]))


In [ ]:
def minima(ndim,N):
    p1 = []
    
    #creates random number array
    for i in range(0,N):
        p1.append(np.random.uniform(-10,10,ndim))
    one_D = []
    for i in p1:
        #calculates minimum at each random starting point
        R = scipy.optimize.minimize(blackbox_func, i, method='BFGS')
        R_x = round(R.fun,3)
        
        #check if calculated minimum is in an array, if not appends to that array
        if R_x not in one_D:
            one_D.append(R_x)
    return len(one_D)

In [ ]:
def find_nminima():
    """
    Your function should return the list or array of 5 integers 
    """
    #At each dimension, create array of random numbers then iterate over finding each minimum
    Final_array = []
    
    Final_array.append(minima(1,100))
    Final_array.append(minima(2,100))
    Final_array.append(minima(3,100))
    Final_array.append(minima(4,500))
    Final_array.append(minima(5,100))
    
    return Final_array

In [ ]:
my_minima = find_nminima()
assert len(my_minima) == 5
assert (my_minima[0]) == 4


# Task 2 (20 pts)

Generate random numbers from a probability distribution that looks like two triangles. 
The PDF f(x) is the following. Use the inverse CDF sampling method.
* f(x) = 0 when x<0
* f(x)= x/2 when 0<x<=1
* f(x)= 1-x/2 when 1<x<=2
* f(x)= 0 when 2<x<=3
* f(x)= (x-3)/2 when 3<x<4
* f(x)= (5-x)/2 when 4<x<5
* f(x)= 0 when x>=5


In [ ]:
def generate_triangles(N):
    """ Return a numpy array with the length N with 
    random numbers following the distribution specified
    """
    #calculated integrals, then graphed them to line up each section so integral constants could be found
    #limits were also found this way
    
    #create list of random y values 
    ys = np.random.uniform(size=N)
    xs = ys * 0
    
    #statements to filter different values of y
    xs[ys<= 0.25] = 2 * np.sqrt(ys[ys<0.25])
    xs[(ys>0.25) & (ys<=0.5)] = 2 - np.sqrt(-4*ys[(ys>0.25) & (ys<=0.5)]+2)
    xs[ys==0.5] = 0.5
    xs[(ys>0.5) & (ys<=0.75)] = 3 + np.sqrt(4 * ys[(ys>0.5) & (ys<=0.75)]-2)
    xs[(ys>0.75) & (ys<=1)] = 5 - 2 * np.sqrt(1-ys[(ys>0.75) & (ys<=1)])
    
    return xs

In [ ]:
# The code will be tested by this 
# the distribution of points should look like two triangles
plt.hist(generate_triangles(100000), bins=200);

# Task 3 (20 pts)

The motion of bodies in the Solar system can be described by the Kepler equation 
$$2 \pi t/T = E - e \sin(E)$$
where e is orbit eccentricity, t is the time of observation, T is the orbital period. And E is the so-called eccentric anomaly. 
If you know the eccentric anomaly E (by solving numerically the previous equation) then the position of the body in the plane of the solar system can be described by 
$$x= a  (\cos E -e) $$
$$y= b \sin E $$
where a, b are the semi-major axis and semi-minor axis respectively.

Your task is to write the function that determines the uncertainty on the position x given the time of observation (t), ecentricity (e), period (T), semi-major axis (a) and their uncertainties.
You can assume that the eccentric anomaly E will be always between -1000 and 1000.
We will verify that the errors are accurate to within 10%

In [ ]:
def kepler_uncert(t=None, T=None, e=None, a=None, err_t=None, err_T=None, err_e=None, err_a=None):
    # return the value of the uncertainty on the x position of the solar system body
    N = 10000
    #create arrays of random numbers from normal distribution
    ts = np.random.normal(t, err_t, size=N)
    Ts = np.random.normal(T, err_T, size=N)
    es = np.random.normal(e, err_e, size=N)
    a_s = np.random.normal(a, err_a, size=N)
    
    Es = []
    #calculate value of E error uing generated random arrays
    for i in range(len(es)):
        kepler = lambda E: E - (es[i] * np.sin(E)) - ((2 * np.pi * ts[i])/Ts[i])
        Es.append(scipy.optimize.brentq(kepler,-1000,1000))
    
    xs = a_s * (np.cos(Es) - es)

    return xs.std()

In [ ]:
print(kepler_uncert(t=1, T=2, e=.5, a=3, err_t=1e-5, err_T=1e-4, err_e=1e-1, err_a=0.01))
# This value should be close to 0.015

print(kepler_uncert(t=3, T=4, e=.4, a=6, err_t=1e-4, err_T=1e-3, err_e=1e-2, err_a=0.02))
# This should be close to 0.106


In [ ]:
print(kepler_uncert(t=1, T=2, e=.5, a=3, err_t=1e-5, err_T=1e-4, err_e=1e-3, err_a=0.01))
# This value should be close to 0.015

# Task 4 (20 pts)

# Fitting a periodic model to the data. 

In the input data ch2_task4.txt. we have repeated measurements from the instrument. 
The file has 3 columns. One is the time of observation, the second one is the measurement, and the third one is the uncertainty of the measurement. 
We have reasons to believe that the measurement periodically change with time, so we want to fit the dataset provided by a periodic model 
$$M(x) = A \sin(2\pi x/T) + B \cos(2\pi x/T) $$
in order to determine the period T from the data. A,B,T are model parameters

Assuming that we know that the period should be between 0.5 and 5, determine the best period describing the data with the accuracy of better than 0.1.

Your function needs to return the tuple with the best period and bestmodel values

Hint: Use linear regression to fit the periodic models for a grid of periods. Select the period providing the best fit.

In [ ]:
T4_DATA = np.loadtxt('ch2_task4.txt')

plt.errorbar(T4_DATA[0], T4_DATA[1], T4_DATA[2], fmt='.')

def findper():
    #shorten data names
    X = T4_DATA[0]
    Y = T4_DATA[1]
    E = T4_DATA[2]
    
    #inital setup for loop
    N = len(X)
    A = np.zeros((N,2))
    Tval = np.linspace(0.5,5,10000)
    T = 0
    kai = 10000
    para = 0
    
    #loop over each value of period to find best using lowest kai squared value
    for i in Tval:
        x1 = np.sin((2*np.pi*X)/i)
        x2 = np.cos((2*np.pi*X)/i)
    
        A[:,0] = x1/E
        A[:,1] = x2/E
        
        #calculate parameter and err, then check against current lowest kai squared value
        p,res,rnk,a = scipy.linalg.lstsq(A,Y/E)
        if res < kai:
            kai = res
            T = i
            para = p
            
    values = para[0] * np.sin((2*np.pi*X)/T) + para[1] * np.cos((2*np.pi*X)/T)
    return (T,values)    

In [ ]:
# Your code will be tested with this 
plt.errorbar(T4_DATA[0], T4_DATA[1], T4_DATA[2], fmt='.')
bestp, bestmod = findper()
print('BEST PERIOD', bestp)
plt.plot(np.sort(T4_DATA[0]), bestmod[np.argsort(T4_DATA[0])])


# Task 5 (20 pts)

You are given a dataset from the particle physics experiment. 
In that experiment the number of particles are recorded in intervals of Energy.
The dataset contains the array of those numbers and the center Energy of each bin/interval. 
We believe that the average number of particles in each bin could be described by  this model 
$$R(E) = A_1+ A_2 \exp(A_3  E) + A_4  \exp
\left( - \frac{1}{2}\frac{(E-A_5)^2}{A_6^2} \right)$$

where the first two terms are describing the background and the last term is a contribution from a decaying 
particle with the mean energy A5. This contribution creates a peak in the energy distribution. Your task is to measure A5.

In the dataset you can assume that the number of particles in each bin is described by Poisson distribution with the rate described by the equantion above. You can assume that measurement of number of particles in each bin are independent from each other.

Write the likelihood function and estimate the energy of the particle $A_5$ using maximum likelihood approach. 
Overplot your best model on top of the data and make sure it fits well. 

Determine the uncertainty of your measurement. Your function solve_task5() should return two numbers
the estimate of the energy and its uncertainty. (If you don't know how to evaluate uncertainty, you can just return np.nan instead)

In [ ]:
T5_DATA = np.loadtxt('ch2_task5.txt', dtype=[('E', np.float64), ('N', int)])
T5_DATA['E']
T5_DATA['N']
plt.plot(T5_DATA['E'], T5_DATA['N'], drawstyle='steps')
plt.xlabel('Energy')
plt.ylabel('Number of particles')

In [ ]:
def like(p,Y):
    
    X = T5_DATA['E']
    A1,A2,A3,A4,A5,A6 = p
    #creating rate for the poisson distribution 
    rate = A1 + (A2 * np.exp(A3 * X)) + (A4 * np.exp(-(X-A5)**2/(2*(A6**2))))
    if rate[1] > 10000:
        rate[:] = 1
    #creating possion ditribution object with above rate
    P = scipy.stats.poisson(rate)
    #calculate pmf of different N values
    prob = P.logpmf(Y)
    
    #log and sum to get loglike
    loglike = prob.sum()

    if not np.isfinite(loglike):
        return 1e10
    
    return -loglike

In [ ]:
def solve_task5():
    """
    Your function needs to return the best period and the uncertainty. 
    It also needs to overplot the best model on top of the data
    
    Inital parameter values were estimated by realising that the shape of the function is similiar to a gaussian:
    A1 is the offset from the x axis
    A2 and A3 need to be near 0 due to no other exp being seen on the graph
    A4 is the height of the gaussian
    A5 is where the gaussian is centred
    A6 is the width of the gaussian
    """
    #shorten data statements
    X = T5_DATA['E']
    Y = T5_DATA['N']
    
    #calculate the best value and err using estimated parameter values
    res = scipy.optimize.minimize(like,[25,0,0,60,70,10],args=(Y,))
    err = np.sqrt(np.diag(res.hess_inv))
    
    #plotting statements
    plt.plot(X,Y,drawstyle = "steps", label = ("Data"))
    xgrid=np.linspace(50,80,60)
    plt.plot(xgrid,res.x[0] + (res.x[1] * np.exp(res.x[2] * X)) + res.x[3] * np.exp(-(X-res.x[4])**2/(2*(res.x[5]**2))),
             label = "Function")
    plt.legend()
    plt.xlabel("Energy")
    plt.ylabel("Number of particles")
    
    return res.x[4],err[4]

In [ ]:
bestval, bessterr = solve_task5()

